# X-Ray: Anomaly-Detection and Classification

### Computational Methods II Final Project

#### Sarah Yam and Joseph Hostyk

In [1]:
%matplotlib inline 
import os
import matplotlib.pyplot as plt 
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import inspect
from torchsummary import summary
from PIL import Image
from collections import defaultdict, Counter
import random
import copy
import pandas as pd



### Load and clean our data:

### Get labels:
def getPatientInfo(patientDataFile):
    
    patientDataDict = defaultdict(dict)
    classificationsToPatients = defaultdict(set)
    
    with open(patientDataFile, "r") as patients:
        header = patients.readline().strip().split(",")
        for patient in patients:
            
            # We're only keeping frontal (78k), leaving out lateral (10k)
            
            if "lateral" not in patient:
            
                patient = dict(zip(header, patient.strip().split(",")))
                ID = patient["Patient ID"]
                path = patient["Path"]
                
                classificationRow = [column for column in patient if patient[column] == "1" and column != "Sum per Row"]

                # Might be 'No Finding' and 'Support Devices'. In which case, we want the last option.
                # Otherwise, there should just be one label. In which case we want the last option.
                try:
                    classification = classificationRow[-1]
                    if len(classificationRow) > 1 and classificationRow != ['No Finding', 'Support Devices']:
                        print(classificationRow)
                    patientDataDict[ID]["Path"] = path
                    patientDataDict[ID]["Classification"] = classification
                    
                    classificationsToPatients[classification].add(ID)

                # The final row of the file is just a tally of the columns.
                except IndexError as e:
                    continue
                    
    return patientDataDict, classificationsToPatients

###### will run tests using my set 0 partition (I made 10 partitions, so each set has about ~8k images)
###### partitions were done based on the last digit in the patient ID [0 - 9], so it's mildly randomized

###### current file path to set0 objects: ./set0/Subsetted/train/patients[0 - 9]
###### file path to set0 patient data: ./Subset0.csv 

###### file path to my actual, full subset: ./SubPatients/train/patients[0 - 9]
###### name of datafile in my directory (I'll push it later today): SubsetPatients.csv

patientDataFile = "Subset0.csv"
#patientDataFile = "SubsetPatients.csv"
allPatientsDataDict, classificationsToPatients = getPatientInfo(patientDataFile)
    

In [264]:
### Get labels:
def getPatientInfo(patientDataFile):
    
    i = 0
    classToIndex = defaultdict(set)
    patientDataDict = {}
    
    with open(patientDataFile, "r") as patients:
        header = patients.readline().strip().split(",")
        
        
        for patient in patients:

        # We're only keeping frontal (78k), leaving out lateral (10k)
        # Want {idx: ID, Path, Classification} per patient

            if "Lateral" not in patient:

                patient = dict(zip(header, patient.strip().split(",")))
                
                idx = i
                ID = patient["Patient ID"]
                path = patient["Path"]
                classificationRow = [column for column in patient if patient[column] == "1" and column != "Sum per Row"]
                
                # Might be 'No Finding' and 'Support Devices'. In which case, we want the last option.
                # Otherwise, there should just be one label. In which case we want the last option.
                try:
                    classification = classificationRow[-1]
                    if len(classificationRow) > 1 and classificationRow != ['No Finding', 'Support Devices']:
                        print(classificationRow)
                    
                    # threw in the index because it's the only other unique identifier aside from Path
                    # using idx instead of ID for class to Patients
                    
                    patientDataDict[i] = ID, path, classification
                    classToIndex[classification].add(idx)
                    
                    # increase index for each new entry being used 
                    i += 1
                    
                # The final row of the file is just a tally of the columns.    
                except IndexError as e:
                    continue
             
             
                
    return patientDataDict, classToIndex


# will run tests using my set 0 partition (I made 10 partitions, so each set has about ~8k images)
# partitions were done based on the last digit in the patient ID [0 - 9], so it's mildly randomized

# current file path to set0 objects: ./set0/Subsetted/train/patients[0 - 9]
# file path to set0 patient data: ./Subset0.csv 

# file path to my actual, full subset: ./SubPatients/train/patients[0 - 9]
# name of datafile in my directory (I'll push it later today): SubsetPatients.csv

patientDataFile = "Subset0.csv"
patientDataDict, classToIndex = getPatientInfo(patientDataFile)

    

### Split our data:

In [320]:
# function to split each disease into 80/20

def splitData(classToIndex):
    
    trainIndices = []
    testIndices = []
    
    trainPercentage = 0.8
    
    for classification, patientsWithThatClassification in classToIndex.items():
        
        total = len(patientsWithThatClassification)
        trainAmount = int(trainPercentage * total)
        
        print("There are {} in {}.".format(total, classification))
        
        hm = list(patientsWithThatClassification)
        
        random.shuffle(hm)
        
        trainIndicesForThisClassification = hm[:trainAmount]
        testIndicesForThisClassification = hm[trainAmount:]
        
        
        trainIndices += trainIndicesForThisClassification
        testIndices += testIndicesForThisClassification
        
    return trainIndices, testIndices


In [321]:
'''
This could be an area of confoundment. Since there's patients with more than one image, 
using patientIDs as the identifier could possibly cause two classifications to map to one patient ID.
It's unlikely, but it's definitely something to eventually consider.
'''



trainIndices, testIndices = splitData(classToIndex)
print("We're left with {} train and {} test.".format(len(trainIndices), len(testIndices)))



There are 308 in Fracture.
There are 6112 in Support Devices.
There are 810 in Atelectasis.
There are 285 in Lung Lesion.
There are 192 in Pneumonia.
We're left with 6164 train and 1543 test.


In [322]:
# index to patient matching

def indexToPatient(indexSet, patientDataDict):

    index_patient = [[0,0]] * len(indexSet)

    for i in range(len(indexSet)):

        patientIndex = indexSet[i]
        matchedID = patientDataDict[patientIndex][0]
        
        index_patient[i] = (patientIndex, matchedID)
        
    return index_patient

trainIndexToPatient = indexToPatient(trainIndices, patientDataDict)
testIndexToPatient = indexToPatient(testIndices, patientDataDict)



In [323]:
### Write them out:
import csv 

with open("trainPatients.txt", "w") as out:
    rowbyrow = csv.writer(out)
    rowbyrow.writerows(trainIndexToPatient)

with open("testPatients.txt", "w") as out:
    rowbyrow = csv.writer(out)
    rowbyrow.writerows(testIndexToPatient)

In [329]:
### Attempt 2 with a pandas dataframe instead

class ImageDataset(Dataset):
    '''Image Label Data as Dataset Object'''
    
    def __init__(self, indices, patientDataFrame):
        '''
        Args:
            patientID (string): ID's for patients within data subset
            patientDataFrame (dataframe): ID, Path, Classification
        
        '''
        self.indices = indices
        self.dataframe = patientDataFrame
        
    def __len__(self):
        
        # add 1 for non-length measurement
        return len(self.indices)
    
    def __getitem__(self, index):
        
        # individualName = self.dataframe.iloc[index,0]?
        
        # get ID, Classification, and Path per entry
        # use path to open corresponding image to entry (NOT corresponding to ID)
        individualName = self.indices[index]
        individualClass = self.dataframe.iloc[individualName]["Classification"]
        individualPath = self.dataframe.iloc[individualName]["Path"]
        
        image = Image.open(individualPath)
        pixelArray = np.array(image)
        
        patientInfo = {"classification": individualClass, "image": pixelArray}
        
        return patientInfo

In [330]:
# dataframe to make referencing easy. Continue utilizing idx as unique indexer

patientDataFrame = pd.DataFrame.from_dict(patientDataDict, orient = 'index')
patientDataFrame.columns = ('ID', 'Path', 'Classification')

train_dataset = ImageDataset(trainIndices, patientDataFrame)
test_dataset = ImageDataset(testIndices, patientDataFrame)

##### So it's really not having it with the dicts apparently. The only attributes that I could determine it had were .names and .patientInfo, which actually only just returned the original dict itself, and isn't easily callable. I'm not entirely sure how the images are getting stored or under what attribut name if any. I'm going to try doing this by creating a pandas from the dict.


class ImageDataset(Dataset):
    """ dataset."""

    def __init__(self, names, allPatientsDataDict):
        """
        Args:
            names (string): names
        """
        self.names = names
        self.patientInfo = allPatientsDataDict

    def __len__(self):
        return len(self.names)

    def __getitem__(self, index):
        
        individualName = self.names[index]
        
        # going to try linking via index
        individualDisease = self.patientInfo[individualName]["Classification"]
        individualPath = self.patientInfo[individualName]["Path"]
        
        image = Image.open(individualPath)
        pixelArray = np.array(image)

        patientInfo = {"classification": individualDisease, "image": pixelArray}

        return patientInfo
    

## SO FUN FACT:
#### All of our images are slightly different sizes and the neural network we built isn't flexible enough to handle that, at least not that I can think of right off the top of my head. We'll either have to resize them via PIL as a "transform" or find a way to dynamically take in images of different sizes.

##### Among other options beyond PIL was a sort of informative stackoverflow link I found: https://stackoverflow.com/questions/41907598/how-to-train-images-when-they-have-different-size

In [333]:
# checking the data
'''
MNIST dataset stucture was a priori split into training and testing. Because we don't have data that's pre-defined like 
that, or anywhere near that structure, we have to call it in a different manner. MNIST already has its labels connected to its images for both training and 
testing, so they already have pre-defined sizes/attributes like train_data.size or train_label.size.  Apparently, the 
only way to do this is by checking the sizes of individual samples. So I'll run through a few here and make sure they're
for the most part consistent (hopefully).
'''

train_length = len(train_dataset)
test_length = len(test_dataset)

print("Train data size:", train_length)
print("Test data size:", test_length)

for i in [4, 13, 69, 420, 666, 888]:
    train_entry = train_dataset[i]
    test_entry = test_dataset[i]
    
    print("Train:", train_entry['classification'])
    print("\t Image shape:", train_entry['image'].shape)
    print("\t Image size:", train_entry['image'].size)
    
    print("Test:", test_entry['classification'])
    print("\t Image shape:", test_entry['image'].shape)
    print("\t Image size:", test_entry['image'].size)
    
'''
No idea if the difference in sizes for the different classifications of the images is going to be a good thing or not.
It might make is easier for the learning, but it alsoc ould be more confounding. Who knows.
'''

Train data size: 6164
Test data size: 1543
Train: Fracture
	 Image shape: (320, 389)
	 Image size: 124480
Test: Fracture
	 Image shape: (320, 320)
	 Image size: 102400
Train: Fracture
	 Image shape: (320, 368)
	 Image size: 117760
Test: Fracture
	 Image shape: (320, 355)
	 Image size: 113600
Train: Fracture
	 Image shape: (320, 390)
	 Image size: 124800
Test: Support Devices
	 Image shape: (320, 390)
	 Image size: 124800
Train: Support Devices
	 Image shape: (320, 390)
	 Image size: 124800
Test: Support Devices
	 Image shape: (320, 389)
	 Image size: 124480
Train: Support Devices
	 Image shape: (320, 390)
	 Image size: 124800
Test: Support Devices
	 Image shape: (320, 389)
	 Image size: 124480
Train: Support Devices
	 Image shape: (320, 390)
	 Image size: 124800
Test: Support Devices
	 Image shape: (320, 390)
	 Image size: 124800


'\nNo idea if the difference in sizes for the different classifications of the images is going to be a good thing or not.\nIt might make is easier for the learning, but it alsoc ould be more confounding. Who knows.\n'

In [346]:
d1 = []
d2 = []
sizes = []

for i in range(len(train_dataset)):
    train_entry = train_dataset[i]
    
    d1.append(train_entry['image'].shape[0])
    d2.append(train_entry['image'].shape[1])
    
    sizes.append(train_entry['image'].size)

print("average d1 size: ", np.average(d1))
print("min:", np.min(d1))
print("max:", np.max(d1))
print("average d2 size: ", np.average(d2))
print("min:", np.min(d2))
print("max:", np.max(d2))
print("average size: ", np.average(sizes))
print("min:", np.min(sizes))
print("max:", np.max(sizes))

average d1 size:  320.1291369240753
min: 320
max: 390
average d2 size:  383.8392277741726
min: 320
max: 486
average size:  122869.87670343932
min: 102400
max: 155520


In [334]:
batch_size = 100
num_epochs = 1876

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### Our model.
#### SOMETHING SOMETHING DENSE NETWORKS?

### Training the model.

### Testing the model.

## Below is the code from the Lab.

In [85]:
batch_size = 100
num_epochs = 1876

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Building a Dynamic CNN
The above code is great for getting an idea of how to build a model. But in production we would want to be able to iterate through multiple different architectural choices like number of layers, filter sizes, pooling kernels, strides, activation functions, etc. In order to be able to do this we need to build our model using arguments which specify the architecture and loops. Otherwise we'd have to explicitly type out every layer of our model, for every change we wanted to implement.

Now just in case any of your are getting flashbacks to lab 4, not to worry. I've implemented most of the function for you. You're encouraged to walk through it to understand what's happening. **All you have to do is implement the method `calculateFinalOutputSize` that calculates the size of the final output, which is needed to calculate the size of the final FCNN layer.**

In [335]:
class CNNModelDynamic(nn.Module):
    def __init__(self, input_shape, n_classes,
                 in_channels_list, out_channels_list,
                 kernel_size_list, stride_list,
                 padding_list, pool_kernel_list,
                 pool_stride_list,
                 pooling_list, activations_list):
        super(CNNModelDynamic, self).__init__()
        localArgs = locals().items()
        argLens = set()
        ignoredArgs = ['self', "__class__", "input_shape", "n_classes"]
        for argName, arg in localArgs:
            if argName not in ignoredArgs:
                argLens.add(len(arg))
        assert len(argLens) == 1, ("mismatch in lengths of arguments."
                                   "All params for each layer must be specified")
        finalOutputSize = self.calculateFinalOutputSize(input_shape,kernel_size_list, stride_list,
                                         padding_list, pool_kernel_list, pool_stride_list)
        modules = list()
        for layerIdx in range(0, argLens.pop()):
            modules.append(nn.Conv2d(in_channels = in_channels_list[layerIdx],
                                 out_channels = out_channels_list[layerIdx],
                                 kernel_size = kernel_size_list[layerIdx],
                                 stride = stride_list[layerIdx],
                                 padding = padding_list[layerIdx]))
            modules.append(activations_list[layerIdx])
            modules.append(pooling_list[layerIdx](kernel_size = pool_kernel_list[layerIdx],
                                                  stride = pool_stride_list[layerIdx]))
        self.convolutions = nn.Sequential(*modules)
        self.finalLayer = nn.Linear(finalOutputSize**2*out_channels_list[-1], n_classes)
        
    def outputFromConvLayer(self, w, k, p, s):
        return (w-k+2*p)/float(s) + 1
    
    def outputFromPoolLayer(self, w, k, s):
        return (w-k)/float(s) + 1
    
    def calculateFinalOutputSize(self, input_shape, kernel_size_list, stride_list,
                                 padding_list, pool_kernel_list, pool_stride_list):
        """
        Calculates the shape of the final output assuming that every conv layer is followed
        by a pooling layer.
        """
         #### your code here ###
        currentInput = input_shape
        for i in range(len(kernel_size_list)):
            currentInput = self.outputFromConvLayer(currentInput, kernel_size_list[i], padding_list[i], stride_list[i])
            currentInput = self.outputFromPoolLayer(currentInput, pool_kernel_list[i], pool_stride_list[i])
        finalOutputShape = currentInput
        
#         print("Final shape is", int(finalOutputShape))
        
         #### end code here ###
        return(int(finalOutputShape))
        
        
    def forward(self, x):
        out = self.convolutions(x)
        out = out.view(out.size(0), -1)
        out = self.finalLayer(out)
        return(out)

### Dynamic Model Summary
Using the summary function from torchsummary print out the layers, shapes, and number of parameters in the model.

In [336]:
#### your code here ###
cnnDynamicOG = CNNModelDynamic(input_shape = 28, n_classes = 10,
                 in_channels_list = [1, 16], out_channels_list = [16, 32],
                 kernel_size_list = [5, 5], stride_list = [1, 1],
                 padding_list = [0,0], pool_kernel_list = [2, 2],
                 pool_stride_list = [2, 2],
                 pooling_list = [nn.MaxPool2d, nn.MaxPool2d], activations_list = [nn.ReLU(), nn.ReLU()])
summary(cnnDynamicOG, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 24, 24]             416
              ReLU-2           [-1, 16, 24, 24]               0
         MaxPool2d-3           [-1, 16, 12, 12]               0
            Conv2d-4             [-1, 32, 8, 8]          12,832
              ReLU-5             [-1, 32, 8, 8]               0
         MaxPool2d-6             [-1, 32, 4, 4]               0
            Linear-7                   [-1, 10]           5,130
Total params: 18,378
Trainable params: 18,378
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.19
Params size (MB): 0.07
Estimated Total Size (MB): 0.27
----------------------------------------------------------------


## Training a Dynamic CNN
Now we can train our model. First train the original model implemented above. It shoud train just fine and achieve decent accuracy. 

1. Your first task is to implement early stopping by keeping track of your best accuracy and stopping training if accuracy doesn't improve after 3 checks. I suggest that you check your evaluation accuracy every 50 iterations unless you have a GPU then by all means check whenever you want. Hell, check twice an iteration. Ain't nothing stopping you with a GPU. Once your best model is found, quit out and stop running. This is where you'd usually save a model, but don't worry about doing that.

2. Your second task is to experiment with different configurations. Try at least two architectural changes (depth, convolution layers, channels, strides, **optimization**, average pooling, etc) and record some observations about model performance for your two configurations. I would suggest using Adam as an optimization function.

In [337]:
def doTheThing(model, optimizer):
    numIterations = 0
    bestAccuracy = 0
    patience = 2 # how many times should we be ok with our accuracy not increasing?
    checksWithoutIncrease = 0
    num_epochs = 5 # Change later
    for epoch in range(num_epochs):
        print("Epoch: {}".format(epoch))
        for i, (images, labels) in enumerate(train_loader):
            # Load images
            images = images.requires_grad_()

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = model.forward(images)

            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            numIterations += 1
            if numIterations % 50 == 0:
                correct = 0
                total = 0
                # Iterate through test dataset
                for images, labels in test_loader:
                    # Load images
                    images = images.requires_grad_()

                    # Forward pass only to get logits/output
                    outputs = model.forward(images)

                    # Get predictions from the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Total number of labels
                    total += labels.size(0)

                    # Total correct predictions
                    correct += (predicted == labels).sum()

                accuracy = 100 * correct / total
                # Check if early stopping criteria are met
                #### your code here ###

                if accuracy > bestAccuracy:
                    bestAccuracy = accuracy
                    checksWithoutIncrease = 0
                else:
                    checksWithoutIncrease += 1
                    print("\tGone {} rounds without increasing:".format(checksWithoutIncrease))

                #### end code here ###
                print('\tIteration: {}. Loss: {}. Testing Accuracy: {}'.format(numIterations, loss.item(), accuracy))
        # remember, early stopping is qutting out of all training.
        #### your code here ###
                if checksWithoutIncrease > patience:
                    print("We did not increase accuracy over the past 3 rounds. Quitting!")
                    return bestAccuracy, numIterations * (epoch + 1)
        #### end code here ###

In [339]:
cnnDynamicOG = CNNModelDynamic(input_shape = 28, n_classes = 10,
                 in_channels_list = [1, 16], out_channels_list = [16, 32],
                 kernel_size_list = [5, 5], stride_list = [1, 1],
                 padding_list = [0,0], pool_kernel_list = [2, 2],
                 pool_stride_list = [2, 2],
                 pooling_list = [nn.MaxPool2d, nn.MaxPool2d], activations_list = [nn.ReLU(), nn.ReLU()])

# we'll tweak this around
learning_rate = 0.01


optimizerSGD = torch.optim.SGD(cnnDynamicOG.parameters(), lr=learning_rate)
optimizerAdam = torch.optim.Adam(cnnDynamicOG.parameters())

In [340]:
cnnDynamiclargerKernels = CNNModelDynamic(input_shape = 28, n_classes = 10,
                 in_channels_list = [1, 16], out_channels_list = [16, 32],
                 kernel_size_list = [8, 8], stride_list = [1, 1],
                 padding_list = [0,0], pool_kernel_list = [2, 2],
                 pool_stride_list = [2, 2],
                 pooling_list = [nn.MaxPool2d, nn.MaxPool2d], activations_list = [nn.ReLU(), nn.ReLU()])
optimizerSGDlargerKernels = torch.optim.SGD(cnnDynamiclargerKernels.parameters(), lr=learning_rate)

In [341]:
criterion = nn.CrossEntropyLoss()


In [342]:
# bestAccuracyOriginal, totalIterationsOriginal = doTheThing(cnnDynamicOG, optimizerSGD)
# bestAccuracyWithAdam, totalIterationsWithAdam = doTheThing(cnnDynamicOG, optimizerAdam)
bestAccuracyWithLargerKernels, totalIterationsWithLargerKernels = doTheThing(cnnDynamiclargerKernels, optimizerSGDlargerKernels)


Epoch: 0


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 390 and 349 in dimension 2 at /pytorch/aten/src/TH/generic/THTensorMoreMath.cpp:1307

In [ ]:
print("With the original settings, the best accuracy achieved was {}, after {} iterations.".format(bestAccuracyOriginal, totalIterationsOriginal))
print("Using Adam as the optimizer, the best accuracy achieved was {}, after {} iterations.".format(bestAccuracyWithAdam, totalIterationsWithAdam))
print("With a larger kernel size, the best accuracy achieved was {}, after {} iterations.".format(bestAccuracyWithLargerKernels, totalIterationsWithLargerKernels))

